In [ ]:
from novel_swarms.behavior.AngularMomentum import AngularMomentumBehavior
from novel_swarms.behavior.AverageSpeed import AverageSpeedBehavior
from novel_swarms.behavior.GroupRotationBehavior import GroupRotationBehavior
from novel_swarms.behavior.RadialVariance import RadialVarianceBehavior
from novel_swarms.behavior.ScatterBehavior import ScatterBehavior
from novel_swarms.behavior.SensorOffset import GeneElementDifference
from novel_swarms.config.AgentConfig import DiffDriveAgentConfig
from novel_swarms.config.EvolutionaryConfig import GeneticEvolutionConfig
from novel_swarms.config.OutputTensorConfig import OutputTensorConfig
from novel_swarms.config.WorldConfig import RectangularWorldConfig
from novel_swarms.novelty.GeneRule import GeneRule
from novel_swarms.sensors.GenomeDependentSensor import GenomeBinarySensor
from novel_swarms.sensors.SensorSet import SensorSet
from src.generation.evolution import ModifiedHaltingEvolution, ModifiedNoveltyArchieve
from src.generation.halted_evolution import HaltedEvolution
from data.swarmset import SwarmDataset, DataBuilder
import numpy as np
from scipy import ndimage
import cv2
import pygame

def resizeInput(X, w=200):
    frame = X.astype(np.uint8)
    resized = cv2.resize(frame, dsize=(w, w), interpolation=cv2.INTER_AREA)
    return resized

def update_network(network, loss_fn, optim, anchor_image, pos_image, neg_image):
    optim.zero_grad()
    anchor_out, pos_out, neg_out = network.network_with_transforms(anchor_image, pos_image, neg_image)
    loss = loss_fn(anchor_out, pos_out, neg_out)
    loss.backward()
    optim.step()
    return loss

In [ ]:
import numpy as np
import torch
from data.swarmset import ContinuingDataset, SwarmDataset
from src.networks.embedding import NoveltyEmbedding
from src.generation.evolution import ModifiedHaltingEvolution
from src.networks.archive import DataAggregationArchive
from src.hil.HIL import HIL
import time
from src.networks.ensemble import Ensemble

trial_name = f"{str(int(time.time()))}"

TRAIN = True
CLUSTER_AND_DISPLAY = True
WRITE_OUT = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ensemble = Ensemble(size=3, output_size=5, lr=30e-4, learning_decay=0.9, decay_step=4, threshold=9.0, weight_decay=1e-6, new_model=True)
ensemble.load_ensemble("final-original-72")
# ensemble.load_ensemble("full-mini-HIL")
network = ensemble.ensemble[0]
network.eval()

SAVE_CLUSTER_IMAGES = True
SAVE_CLUSTER_MEDOIDS = True
PRETRAINING = True
HUMAN_IN_LOOP = True
SYNTHETIC_HIL = False
EVOLUTION = True
EPOCHS = 20
DATA_SIZE = 10000
EVOLUTIONS = 50
K_CLUSTERS = 8

anchor_dataset = ContinuingDataset("data")
sampled_dataset = SwarmDataset("data/full-mini-dual", rank=0)
evolution, _ = ModifiedHaltingEvolution.defaultEvolver(steps=1200, n_agents=24, evolve_population=100, seed=None)
dataset = anchor_dataset if EVOLUTION else sampled_dataset
random_archive = DataAggregationArchive(scalar=True)

hil = HIL(name=trial_name, synthetic=SYNTHETIC_HIL, data_limiter=DATA_SIZE, clusters=K_CLUSTERS)
HIL_archive = DataAggregationArchive()

for gen in range(EVOLUTIONS):
    # Simulate current population + Save Data
    for i in range(len(evolution.getPopulation())):
        # The collection of the original behavior vector below is only used to collect data to compare with the baseline
        visual_behavior, genome, baseline_behavior = evolution.next()
        dataset.new_entry(visual_behavior, genome, baseline_behavior)

    medoid_acc, cluster_acc = hil.record_medoids(network, dataset)

    # Then, evolve
    start_time = time.time()
    embedded_archive = hil.getEmbeddedArchive(dataset, network)
    evolution.overwriteArchive(embedded_archive, random_archive)
    embedded_behavior = embedded_archive.archive[-evolution.evolve_config.population:]
    evolution.overwriteBehavior(embedded_behavior)
    evolution.saveArchive(trial_name)
    evolution.evolve()
    evolution.restart_screen()

    print(f"Evolution complete for gen{gen}")

    # Cluster current dataset, display clusters, and save for analysis
    if len(dataset) > 0 and SAVE_CLUSTER_IMAGES:
        hil.embed_and_cluster(network, dataset, auto_quit=True)
        evolution.restart_screen()

In [ ]:
from src.ui.clustering_gui import ClusteringGUI
from novel_swarms.novelty.NoveltyArchive import NoveltyArchive

# Cluster over saved behaviors
archive = NoveltyArchive()
for i, (_, genome, behavior, _, _, _, _) in enumerate(anchor_dataset):
    archive.addToArchive(vec=behavior, genome=genome)

agent_config = ConfigurationDefaults.DIFF_DRIVE_AGENT
world_config = ConfigurationDefaults.RECTANGULAR_WORLD
world_config.addAgentConfig(agent_config)
config = ResultsConfig(archive=archive, k_clusters=7, world_config=world_config, tsne_perplexity=20, tsne_early_exaggeration=2, skip_tsne=False)

gui = ClusteringGUI(config)
gui.displayGUI()